In [1]:
library(tidyverse)
source('../../0_metadata/theme_CRP-MISC.R')

solid <- c('Intrahepatic.cholangiocyte','Secretory.cell','Respiratory.ciliated.cell','Ionocyte.luminal.epithelial.cell.of.mammary.gland','Fibroblast.mesenchymal.stem.cell',
           'Respiratory.secretory.cell','Endothelial.cell','Stromal.cell','Adventitial.cell','Gland.cell','Salivary.bronchial.secretory.cell','Pericyte.cell',
           'Pancreatic.Stellate.cell','Pancreatic.alpha.beta.cell','Basal.prostate.cell','Prostate.epithelia','Salivary.gland.cell','Intestinal.enterocyte',
           'Intestinal.secretory.cell','Intestinal.tuft.cell','Type.II.Pneumocyte','Cell.of.skeletal.muscle','Schwann.cell','Tendon.cell','Mesothelial.cell',
          'Plasmablast','Kidney.epithelial.cell')

dist2d <- function(a,b,c) {
 v1 <- b - c
 v2 <- a - b
 m <- cbind(v1,v2)
 d <- det(m)/sqrt(sum(v1*v1))
    return(d)
} 

get_turnover <- function(row){
    x_i <- length(row) -1
    y_i <- length(row)
    
    x <- as.numeric(row[[x_i]])
    y <- as.numeric(row[[y_i]])
    return(dist2d(c(0,0),
                  c(1,1),
                  c(x,y)))
}

color_groups = c('COVID-19' = '#740C33', 'MIS-C' = '#0234C1', 'Control_Non-inflammatory' = '#4b7a47',"MIS-C\nCNH" = '#2BBBB3')

── Attaching packages ───────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
#------------------------
## READ OUTPUTS

wb_meta <- read.delim("../../0_metadata/wbrna.tsv") %>% 
    filter(sample_group_matched != -1) %>% 
    select(PTID_DATE,SEQ_ID..UCSFonly.,Diagnosis,timepoint,origin)

# wb_decon <- read.csv("../../3_wbRNA/deconvolution/prevail_WB_bayes-prism.csv")  %>% 
wb_decon <- read.csv("../../../3_wbRNA/deconvolution_new/prevail_bayes-prism.csv")  %>%
    select(!all_of(solid))

wb_decon[,-1] <- wb_decon[,-1]/rowSums(wb_decon[,-1]) 
wb_decon <- wb_decon %>% 
    reshape2::melt(id.vars="X") %>% 
    rename(wb.ID = X,wb.fraction = value,celltype=variable)


wb_meta <- wb_meta[match(unique(wb_meta$PTID_DATE), wb_meta$PTID_DATE),]



cf_meta <-read.delim("../../0_metadata/cfrna.tsv") %>% filter(passQC_1yes_.1no == 1) %>% 
    filter(sample_group_matched != -1) %>% 
    select(PTID_DATE,cfrna_sample_id)

cf_decon <- read.csv("../../1_sample-output/cfrna_deconvolution.csv") %>% 
    select(!all_of(solid))

cf_decon[,-1] <- cf_decon[,-1]/rowSums(cf_decon[,-1]) 
cf_decon <- cf_decon %>%
    reshape2::melt(id.vars="X") %>% 
    rename(cf.ID = X,cf.fraction = value,celltype=variable) %>%
    filter(!(celltype %in% all_of(solid)))


#------------------------
## CREATE MASTER DATAFRAME
wb_all <- merge(wb_decon,wb_meta,by.y="SEQ_ID..UCSFonly.",by.x="wb.ID")
cfrna_all <- merge(cf_meta,cf_decon,by="cfrna_sample_id",by.y="cf.ID")

all_df <- merge(wb_all,cfrna_all,by=c("PTID_DATE","celltype")) %>% select(cfrna_sample_id,wb.ID,origin,Diagnosis,timepoint,celltype,cf.fraction,wb.fraction)
all_df$cf.fraction <- as.numeric(all_df$cf.fraction) 
all_df$wb.fraction <- as.numeric(all_df$wb.fraction) 

head(all_df,2)


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#------------------------------------------------------------------------


plot_df <- all_df %>% 
    filter(((Diagnosis %in% c("MIS-C","COVID-19")) & (timepoint == "acute")) | Diagnosis == "Control_Non-inflammatory")


plot_df2 <- plot_df %>% 
    mutate(celltype = as.character(celltype)) %>%
    mutate(celltype = ifelse(grepl("Mast|dendritic|Macrophage|B.cell|Basophil|Basal.cell",celltype),"other",celltype))

plot_df2$celltype <- factor(plot_df2$celltype, levels= c('Platelet', 'Erythrocyte.erythroid.progenitor', 'neutrophil', 
                                                'Monocyte', 'NK.Cell', 'Myeloid.progenitor', 'T.Cell', 'Thymocyte','other'))

plot_df2 <- unique(plot_df2)

plot_df2 <- plot_df2 %>% group_by(cfrna_sample_id,Diagnosis,celltype) %>% 
                summarize(cf.fraction = sum(cf.fraction),
                          wb.fraction = sum(wb.fraction))

levels_ct <- levels(plot_df2$celltype)

library(qualpalr)
pal = sample(rev(qualpal(length(levels_ct), colorspace=list(h=c(0,360), s=c(0.3,1), l=c(0.2,0.8)))$hex))


pal = c('#1362AB','#740C33','#FB927F','#B27DF2','#5CFBF4','#B2040D','#F5DB9E','#209510','#DBB008')
       # '#949D0A','#700862','#F5DB9E','#209510','#DBB008','#5C6BFA','#B27DF2','#D6F306','#5EFE36','#A9DAC9','#B2040D','#203760','#4D5D0D','#4A1E80','#F63DFA','#F8169B','#175738','#0C8559','#A1506B','#861FF1','#2BBBB3','#5E210C','#36B976','#CAA376','#D75E16','#73480D','#39F9AD','#85D2F8','#65A2FC','#FC092B','#94A514','#AEBBE6')


names(pal) <- levels_ct


#------------------------------------------------------------------------
#------------------------------------------------------------------------
#------------------------------------------------------------------------

WIDTH  = 5.3
HEIGHT = 2.22

pdf(file=paste0("plots/panelE_turnover.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

plt <- plot_df2 %>% 
    ggplot(aes(x=cf.fraction,y=wb.fraction,color=celltype))+
    geom_point()+
#     geom_abline(intercept = 0)+
    scale_color_manual(values=pal)+
    facet_wrap(~Diagnosis)+
    theme_prevail()+
    labs(x= "Cell-Free Fraction", y= "Whole-Blood Fraction")+
    theme(strip.background = element_blank(),
          legend.position = "top",
          legend.spacing.x = unit(.001, 'cm'))+
    guides(color=guide_legend(nrow=2,byrow=TRUE))+ #,override.aes = list(size=3)
    ylim(0,1)

legend <- cowplot::get_legend(plt)

plt + theme(legend.position = "none")

dev.off()

#---------------------------
WIDTH  = 6.94
HEIGHT =1

pdf(file=paste0("plots/panelE_legend.pdf"),
    width=WIDTH,height=HEIGHT, paper="special", bg="white",
    fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)

legend <- gridExtra::grid.arrange(legend)
print(legend)

dev.off()

,cfrna_sample_id,wb.ID,origin,Diagnosis,timepoint,celltype,cf.fraction,wb.fraction
,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>,<dbl>
1,prevail_cu_cfrna_199,PV351,CNH,MIS-C,acute,B.cell,0.023891008,0.042064053
2,prevail_cu_cfrna_199,PV351,CNH,MIS-C,acute,Basal.cell,0.003413765,0.000014149


`summarise()` has grouped output by 'cfrna_sample_id', 'Diagnosis'. You can override using the `.groups`
argument.


png 
  2

TableGrob (1 x 1) "arrange": 1 grobs
  z     cells    name              grob
1 1 (1-1,1-1) arrange gtable[guide-box]


png 
  2